<div style="background:#5D6D7E;padding:20px;color:#ffffff;margin-top:10px;">

# NLP - Práctica 2 ( Recuperación de Información) 

## Profesora: Lisibonny Beato
### Período 3-2023-2024</div>


In [6]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from os import path

<div style="background:#ff6242;padding:20px;color:#ffffff;margin-top:10px;">
<b>El propósito de esta asignación es que el estudiante ponga en práctica la construcción de sistemas de recuperación de información basado en el modelo clásico de RI Vector Space Model, así como también que evalue la efectividad de estos modelos mediante el uso de una colección de referencia (Benchmark).
<br />
<br />
Para esta práctica estará utilizando el siguiente repositorio:https://github.com/oussbenk/cranfield-trec-dataset. En el mismo encontrarán los archivos  cran.all.1400.xml, cran.qry.xml, cranqrel.trec.txt:
<ul>
<li>cran.all.1400.xml contiene 1,400 resumenes de artículos científicos.</li>
<li>cran.qry.xml 225 términos que representan consultas.</li>
<li>cranqrel.trec.txt contiene los juicios de relevancia a dichas consultas.</li>
    </ul>  
<br />
<br />
Estudie en detalle la estructura y el contenido de este conjunto de documentos provistos antes de comenzar.    
<br />
<br />
En este trabajo, aparte del código, debe proveer una interpretación para cada tarea y un análisis para cada resultado obtenido que así lo amerite.</b>
</div>


## 1. Ejercicio 1
### Puntuación máxima de la tarea: 3 puntos
#### Limpieza y preparación de los datos, utilizando distintas técnicas de las ya vistas en clases. Para esta tarea utilizará el archivo cran.all.1400.xml, específicamente sus columnas title y text.


In [7]:
import xml.etree.ElementTree as ElementTree

with open('cranfield-trec-dataset/cran.all.1400.xml', 'r') as f:   # Reading file
    xml = f.read()

xml = '<root>' + xml + '</root>'   # Let's add a root tag

root = ElementTree.fromstring(xml)

# Simple loop through each document
df = pd.DataFrame(columns=['docno', 'title', 'author', 'bib', 'text'])

for doc in root:
    docno = doc.find('docno').text.strip() if doc.find('docno').text is not None else ''
    title = doc.find('title').text.strip() if doc.find('title').text is not None else ''
    author = doc.find('author').text.strip() if doc.find('author').text is not None else ''
    bib = doc.find('bib').text.strip() if doc.find('bib').text is not None else ''
    text = doc.find('text').text.strip() if doc.find('text').text is not None else ''

    new_row = pd.DataFrame({
        'docno': [docno],
        'title': [title],
        'author': [author],
        'bib': [bib],
        'text': [text]
    })
    df = pd.concat([df, new_row], ignore_index=True)

df

,docno,title,author,bib,text
0,1,experimental investigation of the aerodynamics...,"brenckman,m.","j. ae. scs. 25, 1958, 324.",experimental investigation of the aerodynamics...
1,2,simple shear flow past a flat plate in an inco...,ting-yili,"department of aeronautical engineering, rensse...",simple shear flow past a flat plate in an inco...
2,3,the boundary layer in simple shear flow past a...,m. b. glauert,"department of mathematics, university of manch...",the boundary layer in simple shear flow past a...
3,4,approximate solutions of the incompressible la...,"yen,k.t.","j. ae. scs. 22, 1955, 728.",approximate solutions of the incompressible la...
4,5,one-dimensional transient heat conduction into...,"wasserman,b.","j. ae. scs. 24, 1957, 924.",one-dimensional transient heat conduction into...
...,...,...,...,...,...
1395,1396,shear buckling of clamped and simply-supported...,"cook,i.t. and rockey,k.c.","aero. quart. 13, 1962, 41.",shear buckling of clamped and simply-supported...
1396,1397,critical shear stress of an infinitely long si...,"stein,m. and fralich,r.w.","naca tn.1851, 1949.",critical shear stress of an infinitely long si...
1397,1398,stability of rectangular plates under shear an...,"way,s.","j. app. mech. 3, 1936, a131.",stability of rectangular plates under shear an...
1398,1399,buckling of transverse stiffened plates under ...,"wang,t.k.","j.app.mech. 3,1947, a269.",buckling of transverse stiffened plates under ...


<div style="background:#FFFFE0;padding:20px;color:#000000;margin-top:10px;">
Utilice esta celda para colocar comentarios en el notebook, cuando lo estime necesario. Copiela varias veces donde considere.
</div>

In [8]:
import spacy
nlp = spacy.load("en_core_web_sm")

nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Error loading punkt_tab: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1020)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1020)>


False

In [9]:
def data_cleaning(df):
    # Convert to lowercase
    df['title'] = df['title'].str.lower()
    df['text'] = df['text'].str.lower()

    # Remove punctuation and special characters
    df['title'] = df['title'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
    df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

    # Tokenization
    df['title'] = df['title'].apply(word_tokenize)
    df['text'] = df['text'].apply(word_tokenize)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    df['title'] = df['title'].apply(lambda x: [word for word in x if word not in stop_words])
    df['text'] = df['text'].apply(lambda x: [word for word in x if word not in stop_words])

    #Lemmatization
    df['title'] = df['title'].apply(lambda x: [token.lemma_ for token in nlp(' '.join(x))])
    df['text'] = df['text'].apply(lambda x: [token.lemma_ for token in nlp(' '.join(x))])

    return df

In [10]:
df = data_cleaning(df)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/cristiandelahoz/nltk_data'
    - '/Users/cristiandelahoz/Developer/nlp/.venv/nltk_data'
    - '/Users/cristiandelahoz/Developer/nlp/.venv/share/nltk_data'
    - '/Users/cristiandelahoz/Developer/nlp/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


## 2. Ejercicio 2
### Puntuación máxima de la tarea: 2 puntos
#### Construir al menos dos modelos de recuperación de información basado en el modelo vectorial, visto en detalle en clases, cada uno con una configuración distinta de textos, tal y como se indica a continuación: solo title y title+text.

## 3. Ejercicio 3
### Puntuación máxima de la tarea: 5 puntos
#### Evaluar los modelos de recuperación de información construidos, mediante la  utilización de la colección de referencia dada (Archivos cran.qry.xml y cranqrel.trec.txt  ).  Seleccione dos métricas ampliamente utilizadas en la evaluación de este tipo de sistemas y analice los resultados para determinar cual es el mejor de los modelos. 